In [1]:
from skimage.io import imread as skimread
from pathlib import Path
import torch
import numpy as np
from tnia.deeplearning.dl_helper import quantile_normalization
from torchvision import transforms
from torchvision.transforms import v2

c:\Users\bnort\miniconda3\envs\pytorch_and_SAM3\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

tnia_images_path = Path("D:/images/tnia-python-images")
parent_path=Path(tnia_images_path / r'imagesc/2024_05_07_ct_scan')

images_path = parent_path / 'images'
patch_path = parent_path / 'patches'
labels_path = parent_path / 'labels'

test_name = r'Cored_NonMachined_rec00000489.bmp'

testim = skimread(images_path  / test_name) 

print(testim.shape)
testim = testim[:, :, 0]
axes = 'YX'

(1944, 1944, 3)


In [3]:
model = torch.load(patch_path / 'model1')
model.to('cuda')

BasicUNet(
  (conv_0): TwoConv(
    (conv_0): Convolution(
      (conv): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (adn): ADN(
        (N): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (D): Dropout(p=0.25, inplace=False)
        (A): ReLU()
      )
    )
    (conv_1): Convolution(
      (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (adn): ADN(
        (N): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (D): Dropout(p=0.25, inplace=False)
        (A): ReLU()
      )
    )
  )
  (down_1): Down(
    (max_pooling): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (convs): TwoConv(
      (conv_0): Convolution(
        (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (adn): ADN(
          (N): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (D)

In [4]:
device = torch.device("cuda")

In [5]:
testim.shape

(1944, 1944)

In [6]:
testim_ = testim.copy().astype(np.float32)

if axes == 'YXC':
    for i in range(1):
        testim_[:,:,i] = quantile_normalization(
            testim[:,:,i],
            quantile_low=0.01,
            quantile_high=0.998,
            clip=True).astype(np.float32)
else:
    testim_ = quantile_normalization(
        testim_,
        quantile_low=0.01,
        quantile_high=0.998,
        clip=True).astype(np.float32)

tensor_transform = transforms.Compose([
    v2.ToTensor(),
])
x = tensor_transform(testim_)
x = x.unsqueeze(0).to(device)
#x = torch.from_numpy(testim_).to(device)

print(x.shape)
y = model(x)


c:\Users\bnort\miniconda3\envs\pytorch_and_SAM3\lib\site-packages\torchvision\transforms\v2\_deprecated.py:41: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.
  warnings.warn(


torch.Size([1, 1, 1944, 1944])


In [7]:
# search for ROIu
import json

labels_image_path = labels_path / 'input0'

# get all jsons in labels_image_path
jsons = list(labels_image_path.glob('*.json'))

test_ = test_name.split('.')[0]

rois=[]

for json_name in jsons:
    # open json
    with open(json_name, 'r') as f:
        json_ = json.load(f)
        print(json_)
        
        print(json_['source_name'])
        if json_['source_name'] == test_name:
            y1= json_['bbox'][0]
            x1= json_['bbox'][1]
            y2= json_['bbox'][2]
            x2= json_['bbox'][3]
            rois.append([[x1, y1], [x2, y2]])
        
print(jsons)


{'source_name': 'Cored_NonMachined_rec00000489.bmp', 'bbox': [105, 570, 972, 1045]}
Cored_NonMachined_rec00000489.bmp
{'source_name': 'Cored_NonMachined_rec00000489.bmp', 'bbox': [951, 1272, 1619, 1791]}
Cored_NonMachined_rec00000489.bmp
[WindowsPath('D:/images/tnia-python-images/imagesc/2024_05_07_ct_scan/labels/input0/Cored_NonMachined_rec00000489_0.json'), WindowsPath('D:/images/tnia-python-images/imagesc/2024_05_07_ct_scan/labels/input0/Cored_NonMachined_rec00000489_1.json')]


In [8]:
import napari
viewer = napari.Viewer()
viewer.add_image(testim, name='testim')
viewer.add_image(y.cpu().detach()[0, 0].numpy(), name='prediction')
binary = y.cpu().detach()[0, 0].numpy() > 0.5
binary = binary.astype(np.uint8)
binary = binary*2
viewer.add_labels(binary, name='prediction binary')
boxes_layer = viewer.add_shapes(
            name="Label box",
            face_color="transparent",
            edge_color="green",
            edge_width=2,
        )

boxes_layer.add(rois)